In [2]:
import sys
import os
sys.path.insert(0,'../classes')
sys.path.insert(0,'../analysis')
import anl

sys.path.insert(0,'/home/zz737/projects/fiar/cog_model/fourinarow/Analysis notebooks/new')
hqfd = '/home/zz737/projects/fiar/cog_model/fourinarow/Analysis notebooks/new/Heuristic quality'
sys.path.insert(0,hqfd)
from fourinarowfunctions import *
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import logging
import numpy as np

import coloredlogs

from arena import Arena
from coach import Coach
from beck.beck_game import BeckGame as Game
from beck.beck_nnet import NNetWrapper as nn
from beck.beck_players import HumanBeckPlayer
from mcts import MCTS
from utils import *
log = logging.getLogger(__name__)

from keras import backend as K

import tournament
# participant_iters = tournament.participant_iters

participants_dir = '/scratch/zz737/fiar/tournaments/tournament_4' # dir for trained networks
results_dir = '/scratch/zz737/fiar/tournaments/results/tournament_4'
moves_dir = '/scratch/zz737/fiar/tournaments/tournament_4/moves/raw/'
splits_dir = '/scratch/zz737/fiar/tournaments/tournament_4/moves/splits/'

import load
import plot_fun
from importlib import reload
reload(load)

<module 'load' from '../analysis/load.py'>

In [3]:
import value_analysis as va
import pandas as pd
import create_database as cd

In [4]:
opt_boards, opt_values=va.load_opt_value_test_boards()

In [5]:
import pandas as pd
import create_database as cd

all_players = pd.read_pickle(cd.DATABASE_LOC)

In [6]:
dir = '/scratch/zz737/fiar/depth/tournament_6_mcts100_cpuct2_id-2'
fn = 'depth_entropy_NsWeightedMeanEntropy_pvcorr_combined.pkl'

res_df_id2 = pd.read_pickle(os.path.join(dir,fn))

In [7]:
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [39]:
reg = smf.ols('depth~iter + entropy + pvcorr+Ns_weighted_mean_entropy', data=res_df_id2).fit_regularized(method='elastic_net',alpha=0.001,L1_wt=1)



In [40]:
reg.params

Intercept                   12.177710
iter                         0.076068
entropy                      0.614711
pvcorr                      -1.000422
Ns_weighted_mean_entropy    -3.960205
dtype: float64

In [34]:
reg.params

array([ 4.90282474,  0.19340808,  0.35165802,  0.29153696, -1.17267574])

In [21]:
res_df_id2['bp_wp'] = res_df_id2['bp'].astype(str)+'_'+res_df_id2['wp'].astype(str)

In [22]:
reg = smf.ols('depth~iter +Ns_weighted_mean_entropy+C(bp_wp)', data=res_df_id2).fit()


# reg.summary()

In [24]:
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  depth   R-squared:                       0.710
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     85.65
Date:                Thu, 23 Dec 2021   Prob (F-statistic):               0.00
Time:                        16:44:35   Log-Likelihood:            -2.6695e+05
No. Observations:              105408   AIC:                         5.398e+05
Df Residuals:                  102472   BIC:                         5.679e+05
Df Model:                        2935                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              15.0118      0.548     27.412      0.000      13.938      16.085
C(bp_wp)[T.100749392_17200875808]       1.3371      0.772      1.731      0.083      -0.177       2.851
C(bp_wp)[T.10105150496_6990373457]     -2.6429      0.772     -3.422      0.001      -4.156      -1.129
C(bp_wp)[T.10200547328_402665472]       1.0310      0.773      1.334      0.182      -0.484       2.546
C(bp_wp)[T.10201595904_402665476]      -0.0792      0.773     -0.102      0.918      -1.594       1.436
C(bp_wp)[T.10204776464_10514472]        2.2576      0.772      2.924      0.003       0.744       3.771
C(bp_wp)[T.10205288468_4583335240]     -0.2238      0.772     -0.290      0.772      -1.737       1.290
C(bp_wp)[T.10205289488_4305494636]     -9.1605      0.772    -11.863      0.000     -10.674      -7.647
C(bp_wp)[T.10205354004_4583335272]     -1.7714      0.772     -2.294      0.022      -3.285      -0.258
C(bp_wp)[T.10218378288_283142217]       0.6360      0.772      0.824      0.410      -0.878       2.149
C(bp_wp)[T.1027_6152]                  -1.8521      0.773     -2.397      0.017      -3.367      -0.338
C(bp_wp)[T.10482115106_6596598872]     -2.6179      0.772     -3.390      0.001      -4.131      -1.104
C(bp_wp)[T.10485761_4315939072]        -2.8682      0.773     -3.711      0.000      -4.383      -1.353
C(bp_wp)[T.10485776_28704]             -4.7743      0.773     -6.178      0.000      -6.289      -3.260
C(bp_wp)[T.1048593_10489856]            1.2230      0.773      1.582      0.114      -0.292       2.738
C(bp_wp)[T.10486016_12328]             -2.7945      0.773     -3.615      0.000      -4.310      -1.279
C(bp_wp)[T.1048601_10489858]           -3.8516      0.773     -4.984      0.000      -5.366      -2.337
C(bp_wp)[T.10486032_28712]             -4.0147      0.773     -5.194      0.000      -5.530      -2.500
C(bp_wp)[T.1048840_1073742913]          1.9377      0.774      2.504      0.012       0.421       3.454
C(bp_wp)[T.10489856_1048601]            2.5773      0.773      3.334      0.001       1.062       4.092
C(bp_wp)[T.10489856_1074792448]        -0.5941      0.773     -0.769      0.442      -2.108       0.920
C(bp_wp)[T.10489856_4202504]            1.4806      0.773      1.916      0.055      -0.034       2.995
C(bp_wp)[T.10489856_4202536]            2.5909      0.773      3.354      0.001       1.077       4.105
C(bp_wp)[T.10489856_4218888]            1.4697      0.773      1.902      0.057      -0.045       2.984
C(bp_wp)[T.10489858_2148532249]        -0.7821      0.772     -1.013      0.311      -2.296       0.732
C(bp_wp)[T.10489872_4284424]            1.3841      0.772      1.792      0.073      -0.130       2.898
C(bp_wp)[T.10489872_5251112]            0.6600      0.773      0.85

In [15]:
depth_reg_iter = smf.ols('depth~iter', data=res_df_id2).fit()
ent_reg_iter = smf.ols('entropy~iter', data=res_df_id2).fit()


In [16]:
depth_reg_iter.resid

0      -12.629635
1       -1.629635
2       -1.629635
3        5.370365
4        5.370365
          ...    
2943     2.783575
2944     5.783575
2945    -2.216425
2946     0.783575
2947     6.783575
Length: 105408, dtype: float64

In [17]:
ent_reg_iter.resid

0       1.266201
1       1.328037
2      -0.468037
3      -0.597791
4       0.070805
          ...   
2943    0.964075
2944   -1.797016
2945    0.910344
2946   -0.773939
2947   -0.792577
Length: 105408, dtype: float64

In [19]:
resid_reg=sm.OLS(depth_reg_iter.resid, sm.add_constant(ent_reg_iter.resid)).fit()

In [21]:
resid_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.126
Model:                            OLS   Adj. R-squared:                  0.126
Method:                 Least Squares   F-statistic:                 1.523e+04
Date:                Thu, 23 Dec 2021   Prob (F-statistic):               0.00
Time:                        15:56:16   Log-Likelihood:            -3.0801e+05
No. Observations:              105408   AIC:                         6.160e+05
Df Residuals:                  105406   BIC:                         6.160e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -5.967e-15      0.014  -4.31e-13      1.000      -0.027       0.027
0             -1.9115      0.015   -123.426      0.000      -1.942      -1.881
==============================================================================
Omnibus:                     5600.228   Durbin-Watson:                   1.813
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10363.830
Skew:                           0.407   Prob(JB):                         0.00
Kurtosis:                       4.303   Cond. No.                         1.12
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
ent_reg_iter.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                entropy   R-squared:                       0.296
Model:                            OLS   Adj. R-squared:                  0.296
Method:                 Least Squares   F-statistic:                 4.426e+04
Date:                Thu, 23 Dec 2021   Prob (F-statistic):               0.00
Time:                        15:57:58   Log-Likelihood:            -1.3777e+05
No. Observations:              105408   AIC:                         2.755e+05
Df Residuals:                  105406   BIC:                         2.756e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.5511      0.005    543.675      0.000       2.542       2.560
iter          -0.0346      0.000   -210.381      0.000      -0.035      -0.034
==============================================================================
Omnibus:                     2815.807   Durbin-Watson:                   1.688
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1563.280
Skew:                          -0.123   Prob(JB):                         0.00
Kurtosis:                       2.457   Cond. No.                         48.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet

ModuleNotFoundError: No module named 'sklearn'